- Fix inconsistencies in parent column


In [577]:
import pandas as pd
import numpy as np
import os

from glob import glob
import yaml
import re
import copy
import json

# Hard Coded stuff


In [578]:
validation_coder = {
    "number": "regex search ([0-9]+\.[0-9]*.?)|([0-9]+)",
    "integer": "regex search ([0-9]+)",
    "string": "",
}

In [579]:
base_cols = [
    "Attribute",
    "Description",
    "Valid Values",
    "DependsOn",
    "Properties",
    "Required",
    "Template",
    "Parent",
    "DependsOn Component",
    "Source",
    "Validation Rules",
]

keep_cols = [
    "Attribute",
    "Description",
    "Valid Values",
    "Required",
    "DependsOn",
    "DependsOn Component",
    "Properties",
    "Validation Rules",
    "Template",
    "Parent",
    "Source",
    "Type",
    "Ontology",
    "multivalue",
]

# Functions


In [580]:
# Clean list columns into single string
def join_strings(string):
    try:
        return ",".join(string)
    except:
        return ""

In [581]:
def search_df(df, pattern):
    mask = np.column_stack(
        [df[col].str.contains(pattern, na=False, flags=re.IGNORECASE)
         for col in df]
    )

    df = df.loc[mask.any(axis=1)]

    with pd.option_context("display.max_colwidth", None):
        display(df)

    return df

In [582]:
def find_row(df, attribute):
    """Get indexes of the dataframe"""
    indexes = df.index[
        df["Attribute"].str.contains(
            "(^" + re.escape(attribute) + "$)", flags=re.IGNORECASE
        )
    ].tolist()
    if len(indexes) != 0:
        return indexes
    else:
        print(attribute)
        return None


def replace_valid_value(df, indexes, regex_dict, attribute):
    """Alter the dataframe valid values with the replacement value"""
    if indexes == None:
        return df

    elif len(indexes) > 0:
        regex_dict = regex_dict[attribute]

        for index in indexes:
            df.loc[index, "Valid Values"] = re.sub(
                **regex_dict, string=df.loc[index, "Valid Values"]
            )

            # print(df.loc[index, 'Valid Values'])
        return df
    else:
        return df


def code_equals_values(df, regex_dict, attribute):
    print("attribute: ", attribute)

    indexes = find_row(df, attribute)

    print("Index: ", indexes)

    # regex = replace_value(df, replacements, indexes)

    # print("Regex: ", regex)
    if indexes == None:
        return df
    else:
        df = replace_valid_value(df, indexes, regex_dict, attribute)
        return df

In [583]:
# Unzip compressed folder if downloaded from Google Drive
# %unzip 'RFC Tables-20230620T181152Z-001.zip'

# Collect RFCs


In [584]:
with open("local_configs/notebook_config.yaml", "r") as f:
    config = yaml.safe_load(f)

In [585]:
# Get all the RFC file paths
file_paths = glob("../_data/RFC Tables/*")
file_paths

['../_data/RFC Tables/EL Assay_ scRNAseq data model.xlsx',
 '../_data/RFC Tables/EL RFC_ Individual_ Human data model.xlsx',
 '../_data/RFC Tables/EL RFC_ Assay_proteomics Data Model.xlsx',
 '../_data/RFC Tables/EL RFC Assay_Whole Genome Sequencing Data Model.xlsx',
 '../_data/RFC Tables/EL RFC Microbiome Data Model.xlsx',
 '../_data/RFC Tables/EL RFC_ Biospecimen_nonHuman data model.xlsx',
 '../_data/RFC Tables/EL RFC_ Individual_ nonHuman data model.xlsx',
 '../_data/RFC Tables/EL RFC Metabolomics Human Data Model.xlsx',
 '../_data/RFC Tables/EL RFC genotyping_assay.xlsx',
 '../_data/RFC Tables/EL RFC_ Biospecimen_human data model.xlsx',
 '../_data/RFC Tables/EL Assay_ bsSeq (bisulfite-seq_WGBS_methylseq_methylomics) data model.xlsx',
 '../_data/RFC Tables/EL Assay_ RNAseq data model.xlsx']

Create Data Model for Schematic


In [586]:
dm = pd.DataFrame()

# parse through files to create complete data model
for fp in file_paths:
    file_name = os.path.basename(fp)

    temp = pd.read_excel(fp)

    # Create file_name column to check
    temp.insert(loc=0, column="file_name", value=file_name)

    # Create new columnn for data model name
    temp.insert(
        loc=1,
        column="dm",
        value=re.sub(
            "\s\s+",
            " ",
            re.sub(
                "_",
                " ",
                re.sub(
                    "(EL)|(RFC)|(\.xlsx)|([Aa]ssay)|([Dd]ata [Mm]odel)", "", file_name
                ),
            ).strip(),
        ),
    )

    dm = pd.concat([dm, temp])

In [587]:
# initial cleaning
dm[["required", "multivalue"]] = (
    dm[["required", "multivalue"]]
    .fillna(False)
    .astype(str)
    .replace({"1.0": True, "0.0": False})
)

dm.fillna("")
dm.reset_index(drop=True, inplace=True)
dm.head()

,file_name,dm,key,description,valid values,required,requires,multivalue,type,concept source ontology,note,Unnamed: 9,Unnamed: 10,ontology,term id
0,EL Assay_ scRNAseq data model.xlsx,scRNAseq,specimenID,Identifying string linked to a particular samp...,n/a (unique to each data contributor),True,"scRNAseq, Biospecimen",False,string,Sage Bionetworks,NaN,NaN,Sage Bionetworks,NaN,NaN
1,EL Assay_ scRNAseq data model.xlsx,scRNAseq,sampleType,The type of sample collected,"Amniotic Fluid,\nAppendix,\nB cell,\nBasophils...",True,scRNAseq,False,string,"Sage Bionetworks,\nImmPort","The sample types are adopted from Uberon, Cell...",NaN,NaN,NaN,NaN
2,EL Assay_ scRNAseq data model.xlsx,scRNAseq,specifySampleType,"If ""other"" is selected list the type of sample",NaN,False,"scRNAseq,\nsampleType = other",False,string,Sage Bionetworks,NaN,NaN,NaN,NaN,NaN
3,EL Assay_ scRNAseq data model.xlsx,scRNAseq,measurementTechnique,The measurement technique describing the assay...,"16S rRNA gene sequencing,\n1D Gel,\n2D Gel,\nA...",True,scRNAseq,False,string,Sage Bionetworks,NaN,NaN,NaN,NaN,NaN
4,EL Assay_ scRNAseq data model.xlsx,scRNAseq,specifyMeasurementTechnique,"If ""other"" is selected list the name of the me...",NaN,False,"scRNAseq,\nmeasurementTechnique = other",False,string,Sage Bionetworks,NaN,NaN,NaN,NaN,NaN


Data model clean up


In [588]:
# collapse presumed ontology columns and join with existing
dm.loc[:, "ontology"] = dm.iloc[:, 11:].bfill(axis=1).iloc[:, 0]

dm["ontology"] = (
    dm[["concept source ontology", "ontology"]]
    .fillna("")
    .apply(lambda x: ",".join([y.strip() for y in x.unique() if len(y) > 0]), axis=1)
)

# if unique values are provided by data contributor then add this note in the ontology

dm["ontology"] = (
    dm.loc[
        dm["valid values"].str.contains(
            "(n/a \(unique to each data contributor\))", na=False
        ),
        "ontology",
    ]
    + ","
    + "Data Contributor"
)

/var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_56852/2830742079.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dm["valid values"].str.contains(


In [589]:
# new ontologies:
ontology_list = []
for i, v in dm[["concept source ontology", "ontology"]].fillna("").iterrows():
    ontology_list.append(",".join(v))

dm["ontology"] = ontology_list
dm["ontology"] = dm["ontology"].str.strip(",")

In [590]:
dm = dm.apply(
    lambda x: x.str.replace(
        pat="\n|(n/a \(unique to each data contributor\))", repl="", regex=True
    ).str.split(","),
    axis=1,
)

dm.head()

,file_name,dm,key,description,valid values,required,requires,multivalue,type,concept source ontology,note,Unnamed: 9,Unnamed: 10,ontology,term id
0,[EL Assay_ scRNAseq data model.xlsx],[scRNAseq],[specimenID],[Identifying string linked to a particular sam...,[],[True],"[scRNAseq, Biospecimen]",[False],[string],[Sage Bionetworks],NaN,NaN,[Sage Bionetworks],"[Sage Bionetworks, Sage Bionetworks, Data Cont...",NaN
1,[EL Assay_ scRNAseq data model.xlsx],[scRNAseq],[sampleType],[The type of sample collected],"[Amniotic Fluid, Appendix, B cell, Basophils, ...",[True],[scRNAseq],[False],[string],"[Sage Bionetworks, ImmPort]","[The sample types are adopted from Uberon, Ce...",NaN,NaN,"[Sage Bionetworks, ImmPort]",NaN
2,[EL Assay_ scRNAseq data model.xlsx],[scRNAseq],[specifySampleType],"[If ""other"" is selected list the type of sample]",NaN,[False],"[scRNAseq, sampleType = other]",[False],[string],[Sage Bionetworks],NaN,NaN,NaN,[Sage Bionetworks],NaN
3,[EL Assay_ scRNAseq data model.xlsx],[scRNAseq],[measurementTechnique],[The measurement technique describing the assa...,"[16S rRNA gene sequencing, 1D Gel, 2D Gel, Arr...",[True],[scRNAseq],[False],[string],[Sage Bionetworks],NaN,NaN,NaN,[Sage Bionetworks],NaN
4,[EL Assay_ scRNAseq data model.xlsx],[scRNAseq],[specifyMeasurementTechnique],"[If ""other"" is selected list the name of the m...",NaN,[False],"[scRNAseq, measurementTechnique = other]",[False],[string],[Sage Bionetworks],NaN,NaN,NaN,[Sage Bionetworks],NaN


In [591]:
# revert lists back to strings
dm = dm.applymap(lambda x: join_strings(x))

# Rename columns with DCA standards
dm_schema_cols = {
    "dm": "Template",
    "key": "Attribute",
    "description": "Description",
    "valid values": "Valid Values",
    "required": "Required",
    "requires": "DependsOn Component",
    "concept source ontology": "Source",
    "ontology": "Ontology",
    "type": "Type",
}

dm = dm.rename(dm_schema_cols, axis=1)

# drop unimportant columns
r = re.compile("Unnamed*", re.IGNORECASE)

# Add additional required columns for DCA
dm["Properties"] = ""
dm["Validation Rules"] = dm["Type"].map(validation_coder)
dm["DependsOn"] = ""
dm["Parent"] = ""
# dm['DependsOn Component'] = ""

dm = dm[keep_cols]

dm.head()

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
0,specimenID,Identifying string linked to a particular samp...,,True,,"scRNAseq, Biospecimen",,,scRNAseq,,Sage Bionetworks,string,"Sage Bionetworks,Sage Bionetworks,Data Contrib...",False
1,sampleType,The type of sample collected,"Amniotic Fluid,Appendix,B cell,Basophils,Bone,...",True,,scRNAseq,,,scRNAseq,,"Sage Bionetworks,ImmPort",string,"Sage Bionetworks,ImmPort",False
2,specifySampleType,"If ""other"" is selected list the type of sample",,False,,"scRNAseq,sampleType = other",,,scRNAseq,,Sage Bionetworks,string,Sage Bionetworks,False
3,measurementTechnique,The measurement technique describing the assay...,"16S rRNA gene sequencing,1D Gel,2D Gel,Array,B...",True,,scRNAseq,,,scRNAseq,,Sage Bionetworks,string,Sage Bionetworks,False
4,specifyMeasurementTechnique,"If ""other"" is selected list the name of the me...",,False,,"scRNAseq,measurementTechnique = other",,,scRNAseq,,Sage Bionetworks,string,Sage Bionetworks,False


In [592]:
# fixing "not listed" columns
dm["Valid Values"] = dm["Valid Values"].str.replace(
    "Genbank common names (not listed for purposes of this RFC)Unknown",
    "Genbank common names (not listed for purposes of this RFC),Unknown",
    regex=False,
)

dm.loc[dm["Valid Values"].str.contains("not listed"), "Valid Values"] = (
    dm.loc[dm["Valid Values"].str.contains("not listed"), "Valid Values"]
    .str.split(")")
    .apply(
        lambda x: ",".join(
            [y.strip(",") for y in x if not bool(re.search("not listed", y))]
        )
    )
)

# QC
dm.loc[dm["Valid Values"].str.contains("not listed", na=False), "Valid Values"]

Series([], Name: Valid Values, dtype: object)

In [593]:
# Dropping measurement technique
dm = dm.drop(
    index=dm.query('Attribute == "measurementTechnique"').index.values
).reset_index(drop=True)

In [594]:
# combine duplicated attributes
dm = dm.groupby("Attribute").agg(lambda x: ",".join(set(x.astype(str)))).reset_index()

# found extra commas in strings at beginning and end
dm = dm.applymap(lambda x: x.strip(","))

In [595]:
# QA Check
dm["Template"].unique()

array(['Metabolomics Human', 'proteomics,Metabolomics Human',
       'Individual Human', 'Biospecimen nonHuman', 'Individual nonHuman',
       'Biospecimen nonHuman,Biospecimen human', 'proteomics',
       'Individual Human,Individual nonHuman',
       'bsSeq (bisulfite-seq WGBS methylseq methylomics)',
       'Microbiome,bsSeq (bisulfite-seq WGBS methylseq methylomics)',
       'Microbiome,Metabolomics Human',
       'Individual Human,Biospecimen human,Biospecimen nonHuman,Individual nonHuman',
       'Whole Genome Sequencing,RNAseq,Microbiome,bsSeq (bisulfite-seq WGBS methylseq methylomics),scRNAseq',
       'scRNAseq,bsSeq (bisulfite-seq WGBS methylseq methylomics),Microbiome,Whole Genome Sequencing',
       'Whole Genome Sequencing,proteomics,RNAseq,Microbiome,bsSeq (bisulfite-seq WGBS methylseq methylomics),Metabolomics Human,scRNAseq',
       'genotyping',
       'scRNAseq,RNAseq,bsSeq (bisulfite-seq WGBS methylseq methylomics),Whole Genome Sequencing',
       'scRNAseq,RNAseq,Wh

In [596]:
display(dm.dtypes)
display(dm.head())
display(dm.Template.unique())

Attribute              object
Description            object
Valid Values           object
Required               object
DependsOn              object
DependsOn Component    object
Properties             object
Validation Rules       object
Template               object
Parent                 object
Source                 object
Type                   object
Ontology               object
multivalue             object
dtype: object

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
0,acquisitionBatchID,"Acquisition batch identifier, provided by the ...",,False,,mass spec metabolmics,,,Metabolomics Human,,Sage Bionetworks,string,Sage Bionetworks,
1,acquisitionBatchSize,The number of samples,,False,,mass spec metabolmics,,,Metabolomics Human,,Sage Bionetworks,string,Sage Bionetworks,
2,acquisitionBatchSizeUnit,The unit of measurement for number of samples ...,"AFU,AI,AU/ml,DK units/ml,bpg/dl,g/l,gm,HAU,IU,...",False,,"mass spec metabolmics,acquisitionBatchSize",,,Metabolomics Human,,Sage Bionetworks,string,Sage Bionetworks,
3,acquisitionMode,The specific aspect of a mass spectrometer met...,"Unknown,Not collected,Not applicable,Not speci...",True,,"mass spec metabolmics,mass spec proteomics",,,"proteomics,Metabolomics Human",,https://www.ebi.ac.uk/ols/ontologies/ms/terms?...,string,https://www.ebi.ac.uk/ols/ontologies/ms/terms?...,False
4,acquisitionSoftware,The name of the acquisition software used,"Unknown,Not collected,Not applicable,Not speci...",True,,"mass spec metabolmics,mass spec proteomics",,,"proteomics,Metabolomics Human",,http://purl.obolibrary.org/obo/MS_1001455,string,http://purl.obolibrary.org/obo/MS_1001455,False


array(['Metabolomics Human', 'proteomics,Metabolomics Human',
       'Individual Human', 'Biospecimen nonHuman', 'Individual nonHuman',
       'Biospecimen nonHuman,Biospecimen human', 'proteomics',
       'Individual Human,Individual nonHuman',
       'bsSeq (bisulfite-seq WGBS methylseq methylomics)',
       'Microbiome,bsSeq (bisulfite-seq WGBS methylseq methylomics)',
       'Microbiome,Metabolomics Human',
       'Individual Human,Biospecimen human,Biospecimen nonHuman,Individual nonHuman',
       'Whole Genome Sequencing,RNAseq,Microbiome,bsSeq (bisulfite-seq WGBS methylseq methylomics),scRNAseq',
       'scRNAseq,bsSeq (bisulfite-seq WGBS methylseq methylomics),Microbiome,Whole Genome Sequencing',
       'Whole Genome Sequencing,proteomics,RNAseq,Microbiome,bsSeq (bisulfite-seq WGBS methylseq methylomics),Metabolomics Human,scRNAseq',
       'genotyping',
       'scRNAseq,RNAseq,bsSeq (bisulfite-seq WGBS methylseq methylomics),Whole Genome Sequencing',
       'scRNAseq,RNAseq,Wh

In [597]:
# Reorder Columns based on DCA Standards
dm = dm.loc[:, keep_cols]

# Clean up

- Cleanup `DependsOn Component`


In [598]:
recoder = {
    "metabolmics": "metabolomics",
    "(mass spec proteomics)": "Proteomics",
    "(mass spec metabolomics)": "Metabolomics Human",
    "(assay_otheruseTreatment? = Yes)": "assay_other, useTreatment? = Yes",
    "OtherUnknown": "Other, Unknown",
    "falseFalseFALSEtrueTrueTRUE": "TRUE, FALSE",
    "Hispanic or latinoEthnicity": "Hispanic or Latino",
    re.compile("Forwardreverse", flags=re.IGNORECASE): "forward,reverse",
    re.compile("singleEndpairedEnd"): "singleEnd, pairedEnd",
    re.compile("(WGS)"): "Whole Genome Sequencing",
    re.compile("\?"): "",
    "Zeiss LSM 980Other": "Zeiss LSM 980,Other",
    "bsSeqsampleType = other": "bsSeq, sampleType = other",
    re.compile(
        "HPO, MONDO, MAXO codes or labels \(not listed for purposes of this RFC\)"
    ): "HPO and MONDO and MAXO codes or labels (not listed for purposes of this RFC)",
    "The Health, Aging, and Body Composition Study \(HealthABC\)": "The Health and Aging and Body Composition Study (HealthABC)",
}

# 'mass spec metabolomics,measurementTechnique = other'
# falseFalseFALSEtrueTrueTRUE

In [599]:
dm = dm.apply(lambda x: x.replace(recoder, regex=True), axis=1)

In [600]:
with pd.option_context("display.max_colwidth", None):
    display(dm[dm["Valid Values"].str.contains("and Body Composition Study")])

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
16,cohort,Name of the cohort the individual belongs to,"Centenarian, US Family, Denmark Family, The Osteoporotic Fractures in Men (MrOS) Study, Study of Osteoporotic Fractures (SOF), The Health and Aging and Body Composition Study (HealthABC), Cardiovascular Health Study (CHS),Other,Unknown,Not collected,Not applicable,Other, Unknown,Not collected,Not applicable",True,,Individual,,,"Individual Human,Individual nonHuman",,Sage Bionetworks,string,Sage Bionetworks,True


In [601]:
with pd.option_context("display.max_colwidth", None):
    display(dm.query('Attribute == "ethnicity"'))

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
37,ethnicity,Ethnicity of individual,"Not Hispanic or Latino, Hispanic or Latino, Prefer not to answer,Other,Unknown,Not collected,Not applicable",True,,Individual,,,Individual Human,,"Sage Bionetworks,https://www.synapse.org/#!Synapse:syn25878249",string,"Sage Bionetworks,https://www.synapse.org/#!Synapse:syn25878249",False


In [602]:
# QA check
dm[dm["DependsOn Component"].str.contains(
    "metabolomics", case=False, na=False)]

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
0,acquisitionBatchID,"Acquisition batch identifier, provided by the ...",,False,,mass spec metabolomics,,,Metabolomics Human,,Sage Bionetworks,string,Sage Bionetworks,
1,acquisitionBatchSize,The number of samples,,False,,mass spec metabolomics,,,Metabolomics Human,,Sage Bionetworks,string,Sage Bionetworks,
2,acquisitionBatchSizeUnit,The unit of measurement for number of samples ...,"AFU,AI,AU/ml,DK units/ml,bpg/dl,g/l,gm,HAU,IU,...",False,,"mass spec metabolomics,acquisitionBatchSize",,,Metabolomics Human,,Sage Bionetworks,string,Sage Bionetworks,
3,acquisitionMode,The specific aspect of a mass spectrometer met...,"Unknown,Not collected,Not applicable,Not speci...",True,,"mass spec metabolomics,Proteomics",,,"proteomics,Metabolomics Human",,https://www.ebi.ac.uk/ols/ontologies/ms/termsi...,string,https://www.ebi.ac.uk/ols/ontologies/ms/termsi...,False
4,acquisitionSoftware,The name of the acquisition software used,"Unknown,Not collected,Not applicable,Not speci...",True,,"mass spec metabolomics,Proteomics",,,"proteomics,Metabolomics Human",,http://purl.obolibrary.org/obo/MS_1001455,string,http://purl.obolibrary.org/obo/MS_1001455,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,specimenID,Identifying string linked to a particular samp...,,True,,"Biospecimen,genotyping ,Biospecimen,microbiome...",,,"Biospecimen nonHuman,Whole Genome Sequencing,p...",,Sage Bionetworks,string,"Sage Bionetworks,Sage Bionetworks,Data Contrib...","False,false"
156,spectrometerFrequency,The frequency at which a spectrometer causes h...,,True,,"mass spec metabolomics,Proteomics",,regex search ([0-9]+\.[0-9]*.)|([0-9]+),"proteomics,Metabolomics Human",,"Sage Bionetworks,Sage Bionetworks,DSLWG",number,"Sage Bionetworks,Sage Bionetworks,DSLWG",False
159,technologyPlatformVersion,"The specific version (application, manufacture...","Other, Unknown,Not collected,Not applicable,No...",True,,"mass spec metabolomics,bsSeq,RNAseq,microbiome...",,,"Whole Genome Sequencing,proteomics,RNAseq,Micr...",,"Sage Bionetworks,http://purl.obolibrary.org/ob...",string,"Sage Bionetworks,http://purl.obolibrary.org/ob...","False,true"
175,vacuumPressure,The recorded vacuum pressure value,,False,,"mass spec metabolomics,hasIonizationSource = Yes",,regex search ([0-9]+\.[0-9]*.)|([0-9]+),Metabolomics Human,,"Sage Bionetworks,Proposed minimum metadata rel...",number,"Sage Bionetworks,Proposed minimum metadata rel...",


## Cleaning other values and equal values

Removing illegal characters


Remove any special characters


In [603]:
dm["Attribute"] = dm["Attribute"].str.replace("\(|\)|\?", "", regex=True)

Clean up equals in depends on


In [604]:
def create_new_value(old_value):
    new_vals = old_value.split("=")
    new_vals = [nv.strip() for nv in new_vals]
    # convert to camel case
    nv = new_vals[1].capitalize() + new_vals[0][0].upper() + new_vals[0][1:]
    return nv

In [605]:
def recode_yes_no(v):
    if v.lower() == "yes":
        return "TRUE"
    elif v.lower() == "no":
        return "FALSE"
    else:
        return v


dm["Valid Values"] = (
    dm["Valid Values"]
    .apply(lambda x: x.split(","))
    .apply(lambda x: ",".join([recode_yes_no(y) for y in x]))
)

In [606]:
# Split list to process other values
# Find the other columns in the data model
others = dm[dm["DependsOn Component"].str.contains("=", na=False)].copy()
others["DependsOn Component Original"] = others["DependsOn Component"].str.split(",")

# Create series of equals values to use for new attributes/ valid values relationship
others["equals_series"] = others["DependsOn Component Original"].apply(
    lambda x: [y for y in x if bool(re.search("=", y))][0]
)

# others["equals_attribute"] = others["equals_series"].apply(create_new_value)

others[["baseAttribute", "equalsValue"]] = (
    others["equals_series"]
    .str.split("=", expand=True)
    .apply(lambda x: [y.strip() for y in x])
    .rename({0: "base_attribute", 1: "equalsValue"}, axis=1)
)

# Deciding to use true and false for all yes/no values
recoder = {
    re.compile("^[Yy]es", flags=re.IGNORECASE): "TRUE",
    re.compile("true", flags=re.IGNORECASE): "TRUE",
    re.compile("^[Nn]o", flags=re.IGNORECASE): "FALSE",
}

others["equalsValue"] = others["equalsValue"].replace(recoder, regex=True)

others["newDescription"] = others[["baseAttribute", "equalsValue"]].apply(
    lambda x: f"When {x[0].strip()} = {x[1].strip()}", axis=1
)

others["equalsAttribute"] = others[["baseAttribute", "equalsValue"]].apply(
    lambda x: f"{x[1].strip()}{x[0].strip()[0].upper()+x[0].strip()[1:]}", axis=1
)
others["DependsOn Component"] = ""
others["Properties"] = "dataProperty"
others.loc[others["equalsValue"] == "other", "Parent"] = "Specification"

In [607]:
equals_df = others.copy()
equals_df = equals_df.drop(columns=["DependsOn", "Description"])

equals_df = equals_df.rename(
    {
        "Attribute": "DependsOn",
        "newDescription": "Description",
        "equalsAttribute": "Attribute",
    },
    axis=1,
)[base_cols]

equals_df["DependsOn Component"] = ""

equals_df["Valid Values"] = ""

equals_df["Properties"] = "ValidValue"

Update base attribute equals values


In [608]:
# Create mapping
temp = others["equals_series"].str.split("=", expand=True)
temp = temp.apply(lambda x: x.str.strip(), axis=1)
temp = temp.rename({0: "base_attribute", 1: "value_to_replace"}, axis=1)
temp["value_to_replace"] = temp["value_to_replace"].str.capitalize()

# new value
temp["new_value"] = others["equalsAttribute"]
temp.reset_index(drop=True, inplace=True)

temp = temp.drop_duplicates()

In [609]:
replacements = {}

for i, x in temp.iterrows():
    ba, vtr, nv = x
    replacements[ba] = {
        "pattern": re.compile("(" + vtr + ")", flags=re.IGNORECASE),
        "repl": nv,
    }

replacements

{'captivityStatus': {'pattern': re.compile(r'(Captive)',
             re.IGNORECASE|re.UNICODE),
  'repl': 'captiveCaptivityStatus'},
 'hasAssayControl': {'pattern': re.compile(r'(Yes)', re.IGNORECASE|re.UNICODE),
  'repl': 'TRUEHasAssayControl'},
 'diagnosisStatus': {'pattern': re.compile(r'(True)',
             re.IGNORECASE|re.UNICODE),
  'repl': 'TRUEDiagnosisStatus'},
 'ethnicity': {'pattern': re.compile(r'(Hispanic or latino)',
             re.IGNORECASE|re.UNICODE),
  'repl': 'Hispanic or LatinoEthnicity'},
 'hasIonizationSource': {'pattern': re.compile(r'(Yes)',
             re.IGNORECASE|re.UNICODE),
  'repl': 'TRUEHasIonizationSource'},
 'msTarget': {'pattern': re.compile(r'(Targeted)', re.IGNORECASE|re.UNICODE),
  'repl': 'TargetedMsTarget'},
 'useReagent': {'pattern': re.compile(r'(Yes)', re.IGNORECASE|re.UNICODE),
  'repl': 'TRUEUseReagent'},
 'acquisitionBatchSizeUnit': {'pattern': re.compile(r'(Other)',
             re.IGNORECASE|re.UNICODE),
  'repl': 'otherAcquisitionB

In [610]:
for attribute in replacements.keys():
    dm = code_equals_values(dm, replacements, attribute)
    print("-" * 20)

attribute:  captivityStatus
Index:  [13]
--------------------
attribute:  hasAssayControl
Index:  [49]
--------------------
attribute:  diagnosisStatus
Index:  [29]
--------------------
attribute:  ethnicity
Index:  [37]
--------------------
attribute:  hasIonizationSource
Index:  [50]
--------------------
attribute:  msTarget
Index:  [71]
--------------------
attribute:  useReagent
Index:  [173]
--------------------
attribute:  acquisitionBatchSizeUnit
Index:  [2]
--------------------
attribute:  batchSizeUnit
Index:  [11]
--------------------
attribute:  controlType
Index:  [19]
--------------------
attribute:  dnaBatchSizeUnit
Index:  [34]
--------------------
attribute:  databaseName
Index:  [25]
--------------------
attribute:  databaseSource
Index:  [26]
--------------------
attribute:  digestionMethod
Index:  [30]
--------------------
attribute:  experimentalBatchSizeUnit
experimentalBatchSizeUnit
Index:  None
--------------------
attribute:  extractionMethod
Index:  [40]
------

/var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_56852/2963712766.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["Attribute"].str.contains(


# Add new attributes


In [611]:
# dm["Properties"] = "dataProperty"
dm.update(others[base_cols])
dm = pd.concat([dm, equals_df], ignore_index=True)

In [612]:
# Do not need
dm["DependsOn Component"] = ""

# Clean up Valid Values and add them as new Attributes


In [613]:
# valid values that contain other
pure_others = dm[
    dm["Valid Values"].str.contains(
        "([Oo]ther$)", flags=re.IGNORECASE, regex=True)
]

/var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_56852/2172924589.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dm["Valid Values"].str.contains(


In [614]:
pure_others.loc[:, "replacement_value"] = pure_others.loc[:, "Attribute"].apply(
    lambda x: "Other" + (x[0].upper() + x[1:])
)

/var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_56852/2918759120.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pure_others.loc[:, "replacement_value"] = pure_others.loc[:, "Attribute"].apply(


In [615]:
regex_dict = {}

pattern = "([Oo]ther$)"

for i, r in pure_others.iterrows():
    regex_dict[r["Attribute"]] = {
        "pattern": re.compile(pattern, flags=re.IGNORECASE),
        "repl": r["replacement_value"],
    }

In [616]:
temp_dict = {}

for i, r in pure_others.iterrows():
    temp_dict[r["Attribute"]] = {
        "pattern": str(re.compile(pattern, flags=re.IGNORECASE)),
        "repl": r["replacement_value"],
    }

json_formatted = json.dumps(temp_dict, indent=4)
print(json_formatted)

{
    "conversionRatioUnits": {
        "pattern": "re.compile('([Oo]ther$)', re.IGNORECASE)",
        "repl": "OtherConversionRatioUnits"
    },
    "experiementalBatchSizeUnit": {
        "pattern": "re.compile('([Oo]ther$)', re.IGNORECASE)",
        "repl": "OtherExperiementalBatchSizeUnit"
    },
    "lensVoltagesUnit ": {
        "pattern": "re.compile('([Oo]ther$)', re.IGNORECASE)",
        "repl": "OtherLensVoltagesUnit "
    },
    "vacuumPressureUnit": {
        "pattern": "re.compile('([Oo]ther$)', re.IGNORECASE)",
        "repl": "OtherVacuumPressureUnit"
    }
}


In [617]:
for attribute in regex_dict.keys():
    dm = code_equals_values(dm, regex_dict, attribute)
    print("-" * 20)

attribute:  conversionRatioUnits
Index:  [21]
--------------------
attribute:  experiementalBatchSizeUnit
Index:  [39]
--------------------
attribute:  lensVoltagesUnit 
Index:  [58]
--------------------
attribute:  vacuumPressureUnit
Index:  [176]
--------------------


/var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_56852/2963712766.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["Attribute"].str.contains(


In [618]:
# Not sure what happened here in the RFCs
dm["Valid Values"] = dm["Valid Values"].str.replace(
    re.escape("(falseFalseFALSEtrueTrueTRUE)"), "TRUE,FALSE", regex=True
)

In [619]:
valid_values = ",".join(dm["Valid Values"])
valid_values = valid_values.split(",")
valid_values = list(np.unique(valid_values))
valid_values = [v.strip() for v in valid_values if len(v) > 0]

In [620]:
# Fuzzy matching to find misspellings
# Fuzzy matching
from thefuzz import fuzz

scores = {}
for v in valid_values:
    scores[v] = {}
    for v2 in valid_values:
        if v == v2:
            next
        else:
            score = fuzz.ratio(v.lower(), v2.lower())
            if score == 100:
                scores[v][v2] = score
    if len(scores[v]) == 0:
        scores.pop(v)

scores

{'FALSE': {'False': 100},
 'False': {'FALSE': 100},
 'Lipid': {'lipid': 100},
 'Not Specified': {'Not specified': 100},
 'Not specified': {'Not Specified': 100},
 'Plasma': {'plasma': 100},
 'Protein': {'protein': 100},
 'Saliva': {'saliva': 100},
 'Serum': {'serum': 100},
 'Sputum': {'sputum': 100},
 'TRUE': {'True': 100},
 'True': {'TRUE': 100},
 'Urine': {'urine': 100},
 'lipid': {'Lipid': 100},
 'plasma': {'Plasma': 100},
 'protein': {'Protein': 100},
 'saliva': {'Saliva': 100},
 'serum': {'Serum': 100},
 'sputum': {'Sputum': 100},
 'urine': {'Urine': 100}}

In [621]:
# create recoding variables off fuzzy matching
new_values_recoded = []
for v in scores.values():
    new_values_recoded.append(list(v.keys())[0].lower())

new_values_recoded = np.unique(new_values_recoded)

recoder_valid_values = []
for nv in new_values_recoded:
    value_add = {re.compile(nv, flags=re.IGNORECASE): nv}
    recoder_valid_values.append(value_add)


recoder_valid_values

[{re.compile(r'false', re.IGNORECASE|re.UNICODE): 'false'},
 {re.compile(r'lipid', re.IGNORECASE|re.UNICODE): 'lipid'},
 {re.compile(r'not specified', re.IGNORECASE|re.UNICODE): 'not specified'},
 {re.compile(r'plasma', re.IGNORECASE|re.UNICODE): 'plasma'},
 {re.compile(r'protein', re.IGNORECASE|re.UNICODE): 'protein'},
 {re.compile(r'saliva', re.IGNORECASE|re.UNICODE): 'saliva'},
 {re.compile(r'serum', re.IGNORECASE|re.UNICODE): 'serum'},
 {re.compile(r'sputum', re.IGNORECASE|re.UNICODE): 'sputum'},
 {re.compile(r'true', re.IGNORECASE|re.UNICODE): 'true'},
 {re.compile(r'urine', re.IGNORECASE|re.UNICODE): 'urine'}]

# Cleanup valid values


In [622]:
def clean_list(x):
    try:
        result = ",".join([y.strip() for y in x])
        return result
    except:
        return ""

In [623]:
# hard coded dictionary
recoder_valid_values = {
    re.compile("Not Specified", flags=re.IGNORECASE): "Not Specified",
    re.compile("(Other$)", flags=re.IGNORECASE): "Other",
    re.compile("lipid", re.IGNORECASE): "Lipid",
    re.compile("plasma", re.IGNORECASE): "Plasma",
    re.compile("protein", re.IGNORECASE): "Protein",
    re.compile("saliva", re.IGNORECASE): "Saliva",
    re.compile("serum", re.IGNORECASE): "Serum",
    re.compile("sputum", re.IGNORECASE): "Sputum",
    re.compile("urine", re.IGNORECASE): "Urine",
    re.compile(
        "(^0x Visium Spatial Gene Expression)"
    ): "10x Visium Spatial Gene Expression",
    re.compile("falseFalseFALSEtrueTrueTRUE	"): "TRUE, FALSE",
    re.compile("TRUE|TRUEDiagnosisStatus", re.IGNORECASE): "TRUE",
    re.compile("TRUEDiagnosisStatus", re.IGNORECASE): "TRUE",
    re.compile("FALSE", re.IGNORECASE): "FALSE",
    re.compile("$f^", re.IGNORECASE): "F",
    re.compile("UnknownNot collected"): "Unknown, Not collected",
    re.compile(r"\u200b\u200b"): "",
    re.compile(
        "The Health,Aging,and Body Composition Study \(HealthABC\)"
    ): "The Health and Aging and Body Composition Study (HealthABC)",
    re.compile("Not Hispanic or latinoEthnicity"): "Not Hispanic or latino",
    re.compile("Hispanic or latinoEthnicity"): "Hispanic or latino",
    re.compile(
        "HPO, MONDO, MAXO codes or labels \(not listed for purposes of this RFC\)"
    ): "HPO and MONDO and MAXO codes or labels (not listed for purposes of this RFC)",
}

In [624]:
dm["Valid Values"] = (
    dm["Valid Values"]
    .replace(recoder_valid_values, regex=True)
    .str.split(",")
    .apply(lambda x: clean_list(x))
)

valid_values = list(np.unique(",".join(dm["Valid Values"]).split(",")))

valid_values = [v.strip() for v in valid_values if len(v) > 0]

valid_values_df = pd.DataFrame({"Attribute": pd.Series(valid_values)})

valid_values_df["Properties"] = "validValue"
valid_values_df["Required"] = "False"

valid_values_df = valid_values_df[
    ~valid_values_df["Attribute"].isin(dm["Attribute"].tolist())
]

valid_values_df

,Attribute,Properties,Required
0,0x Visium Spatial Gene Expression,validValue,False
1,10x,validValue,False
2,10x Visium Spatial Gene Expression,validValue,False
3,AFU,validValue,False
4,AI,validValue,False
...,...,...,...
406,uiu/ml,validValue,False
407,ul,validValue,False
408,umol/l,validValue,False
409,units/ml,validValue,False


In [625]:
r = re.compile("(^Other)")

measurement_units = np.unique(
    ",".join(
        dm.loc[
            dm["Valid Values"].str.contains("units", regex=True), "Valid Values"
        ].values.tolist()
    ).split(",")
)
measurement_units = [
    x
    for x in measurement_units
    if x not in ["Not Specified", "Other", "Unknown", "Not Available"]
    and not bool(r.search(x))
]

In [626]:
# adding valid values found in attribute columns
dm2 = pd.concat([dm, valid_values_df], axis=0, ignore_index=True)

In [627]:
# Create measurement unit attributes
dm2.loc[dm2["Attribute"].isin(measurement_units), "Parent"] = "MeasurementUnit"
dm2.loc[dm2["Attribute"].isin(measurement_units),
        "Description"] = "Measurement unit"
dm2.loc[dm2["Attribute"].isin(measurement_units), "Type"] = "STRING"
dm2.loc[dm2["Attribute"].isin(measurement_units), "multivalue"] = False

In [628]:
dm2.loc[
    dm2["Valid Values"].str.contains("10x Visium Spatial Gene Expression", na=False),
]

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
77,platformLocation,"The name of the laboratory, facility, vendor, ...","otherPlatformLocation,Unknown,Not collected,No...",True,,,,,"Whole Genome Sequencing,proteomics,RNAseq,Micr...",,"Sage Bionetworks,Sage Bionetworks,DSLWG",string,"Sage Bionetworks,Sage Bionetworks,DSLWG","False,false"
159,technologyPlatformVersion,"The specific version (application, manufacture...","otherTechnologyPlatformVersion,Unknown,Not col...",True,,,,,"Whole Genome Sequencing,proteomics,RNAseq,Micr...",,"Sage Bionetworks,http://purl.obolibrary.org/ob...",string,"Sage Bionetworks,http://purl.obolibrary.org/ob...","False,true"


In [629]:
with pd.option_context("display.max_colwidth", None):
    display(dm2.loc[dm2["Attribute"].str.contains("Possible values"),])

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
465,Possible values are listed under modification parameters,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,Possible values are listed under the cleavage agent nameotherCleavageAgents,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,Possible values are listed under the instrument model term.otherMsInstrumentModel,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [630]:
# Nonsense attributes
dm2 = dm2.drop(
    index=dm2.loc[dm2["Attribute"].str.contains(
        "Possible values"),].index.tolist()
)

In [631]:
vv_recoder = {
    "Possible values are listed under the instrument model term.OtherMsInstrumentModel": "Other",
    "Possible values are listed under the cleavage agent nameOtherCleavageAgents": "Other",
    "Possible values are listed under modification parameters": "",
}

In [632]:
dm2[["Attribute", "Valid Values"]] = dm2[["Attribute", "Valid Values"]].apply(
    lambda x: x.replace(vv_recoder, regex=True)
)

In [633]:
# Fuzzy matching to find misspellings
# Fuzzy matching

from thefuzz import fuzz

valid_values = dm2["Attribute"].replace(
    recoder_valid_values, regex=True).tolist()

scores = {}
for v in valid_values:
    scores[v] = {}
    for v2 in valid_values:
        if v == v2:
            next
        else:
            score = fuzz.ratio(v.lower(), v2.lower())
            if score == 100:
                scores[v][v2] = score
    if len(scores[v]) == 0:
        scores.pop(v)

scores

{'f': {'F': 100}, 'F': {'f': 100}}

In [634]:
with pd.option_context("display.max_colwidth", None):
    display(dm2[dm2["Attribute"] == "f"])

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
41,f,"The data file provides the name of the raw file generated by the instrument. The data files can be instrument raw files but also converted peak lists such as mzML, MGF or result files like mzIdentML.","Unknown,Not collected,Not applicable,Not Specified",True,,,,,Metabolomics Human,,Proteomics Sample Metadata,string,Proteomics Sample Metadata,


In [635]:
with pd.option_context("display.max_colwidth", None):
    display(dm2.loc[dm2["Valid Values"].str.contains("f", na=False),])

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
2,acquisitionBatchSizeUnit,The unit of measurement for number of samples in a batch,"AFU,AI,AU/ml,DK units/ml,bpg/dl,g/l,gm,HAU,IU,iu/l,IU/ml,Kallikrein Inactivator Unit per Milliliter,kg,l,M,mg,mg/dl,mg/l,mg/ml,miu/ml,ml,mM,MOI,ng,ng/dl,ng/ml,ng/nl,ng/ul,nl,nM,Not Specified,NPX,optical density,PFU,PFUe,pg,pg/mg creatinine,pg/ml,pg/nl,pg/ul,pl,pM,Pound,TCID50,ug,ug/dl,ug/l,ug/ml,ug/ul,uiu/ml,ul,uM,umol/l,units/ml,otherAcquisitionBatchSizeUnit",False,,,,,Metabolomics Human,,Sage Bionetworks,string,Sage Bionetworks,
3,acquisitionMode,"The specific aspect of a mass spectrometer method by which mass ranges are selected and possibly dissociated (full scan, MSn, SIM, MRM, etc.).","Unknown,Not collected,Not applicable,Not Specified",True,,,,,"proteomics,Metabolomics Human",,https://www.ebi.ac.uk/ols/ontologies/ms/termsiri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMS_1003213,string,https://www.ebi.ac.uk/ols/ontologies/ms/termsiri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FMS_1003213,False
4,acquisitionSoftware,The name of the acquisition software used,"Unknown,Not collected,Not applicable,Not Specified",True,,,,,"proteomics,Metabolomics Human",,http://purl.obolibrary.org/obo/MS_1001455,string,http://purl.obolibrary.org/obo/MS_1001455,False
11,batchSizeUnit,The unit of measurement for number of samples in a batch,"AFU,AI,AU/ml,DK units/ml,bpg/dl,g/l,gm,HAU,IU,iu/l,IU/ml,Kallikrein Inactivator Unit per Milliliter,kg,l,M,mg,mg/dl,mg/l,mg/ml,miu/ml,ml,mM,MOI,ng,ng/dl,ng/ml,ng/nl,ng/ul,nl,nM,Not Specified,NPX,optical density,PFU,PFUe,pg,pg/mg creatinine,pg/ml,pg/nl,pg/ul,pl,pM,Pound,TCID50,ug,ug/dl,ug/l,ug/ml,ug/ul,uiu/ml,ul,uM,umol/l,units/ml,otherBatchSizeUnit",False,,,,,Metabolomics Human,,Sage Bionetworks,string,Sage Bionetworks,
15,cleavageAgents,Name(s) of the enzyme used,"Possible values are listed under the cleavage agent nameotherCleavageAgents,Unknown,Not collected,Not applicable,Not Specified",True,,,,,proteomics,,Proteomics Sample Metadata,string,Proteomics Sample Metadata,True
16,cohort,Name of the cohort the individual belongs to,"Centenarian,US Family,Denmark Family,The Osteoporotic Fractures in Men (MrOS) Study,Study of Osteoporotic Fractures (SOF),The Health and Aging and Body Composition Study (HealthABC),Cardiovascular Health Study (CHS),Other,Unknown,Not collected,Not applicable,Other,Unknown,Not collected,Not applicable",True,,,,,"Individual Human,Individual nonHuman",,Sage Bionetworks,string,Sage Bionetworks,True
19,controlType,Control samples suitable for normalization and batch correction,"GIS,GoldenWest,NIST SRM 1950,AIBL pool,Baker pool,water plus Baker ISTD,study pool,internal standards,Other,Unknown,Not collected,Not applicable,Not Specified",False,,,dataProperty,,Metabolomics Human,,https://doi.org/10.1101/2020.05.19.105197https://doi.org/10.1038/sdata.2017.140https://www-s.nist.gov/srmors/view_detail.cfmsrm=1950Sage BionetworksDSLWG,string,https://doi.org/10.1101/2020.05.19.105197https://doi.org/10.1038/sdata.2017.140https://www-s.nist.gov/srmors/view_detail.cfmsrm=1950Sage BionetworksDSLWG,
21,conversionRatioUnits,The units of measurement.,"AFU,AI,AU/ml,DK units/ml,bpg/dl,g/l,gm,HAU,IU,iu/l,IU/ml,Kallikrein Inactivator Unit per Milliliter,kg,l,M,mg,mg/dl,mg/l,mg/ml,miu/ml,ml,mM,MOI,ng,ng/dl,ng/ml,ng/nl,ng/ul,nl,nM,Not Specified,NPX,optical density,PFU,PFUe,pg,pg/mg creatinine,pg/ml,pg/nl,pg/ul,pl,pM,Pound,TCID50,ug,ug/dl,ug/l,ug/ml,ug/ul,uiu/ml,ul,uM,umol/l,units/ml,OtherConversionRatioUnits",True,,,,,bsSeq (bisulfite-seq WGBS methylseq methylomics),,Sage Bionetworks,string,Sage Bionetworks,false
23,dataFile,"The data file provides the name of the raw file generated by the instrument. The data files can be instrument raw files but also converted peak lists such as mzML, MGF or result files like mzIdentML.","Unknown,Not collected,Not applicable,Not Specified",True,

In [636]:
dm2[["Attribute", "DependsOn"]] = dm2[["Attribute", "DependsOn"]].apply(
    lambda x: x.replace(recoder_valid_values, regex=True)
)

In [637]:
dm2[
    dm2.duplicated(subset=["Attribute", "Parent", "DependsOn"], keep=False)
].sort_values(by=["Parent", "Attribute"])

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
239,10x Visium Spatial Gene Expression,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,10x Visium Spatial Gene Expression,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Check columns for Speical Characters


In [638]:
check_cols = ["Attribute"]

mask = np.column_stack(
    [dm2[col].str.contains("\(|\)", na=False) for col in dm2[check_cols]]
)

with pd.option_context("display.max_colwidth", None):
    display(dm2[check_cols].loc[mask.any(axis=1)])

,Attribute
279,Cardiovascular Health Study (CHS)
283,Cerebrospinal Fluid (CSF)
350,Illumina Infinium MethylationEPIC BeadChip v1.0 (850k)
351,Illumina Infinium MethylationEPIC BeadChip v2.0 (935k)Illumina MiSeq
398,MassBank of North America (MoNA)
502,Study of Osteoporotic Fractures (SOF)
513,The Health and Aging and Body Composition Study (HealthABC)
514,The Osteoporotic Fractures in Men (MrOS) Study


# Create Manifests in data model


In [639]:
unique_templates = np.unique(
    ",".join(dm2["Template"].dropna().values.tolist()).split(",")
)
templates = pd.DataFrame({"Attribute": unique_templates})
templates["Required"] = "True"
templates["Properties"] = "dataType"
templates["Parent"] = "Template"
templates["Description"] = templates["Attribute"].apply(
    lambda x: f"Metadata template for {x}"
)

# Adjust bseq
templates.loc[
    templates["Attribute"] == "bsSeq (bisulfite-seq WGBS methylseq methylomics)",
    "Description",
] = "Template for bisulfite-seq WGBS methylseq methylomics"
templates.loc[
    templates["Attribute"] == "bsSeq (bisulfite-seq WGBS methylseq methylomics)",
    "Attribute",
] = "bsSeq"

# update templates column for other attributes
dm2["Template"] = dm2["Template"].str.replace(
    "bsSeq (bisulfite-seq WGBS methylseq methylomics)", "bsSeq", regex=False
)

# create depends On column for templates
for i, m in enumerate(templates["Attribute"]):
    dm2["Template"].str.contains(m)
    templates.loc[i, "DependsOn"] = ",".join(
        dm2.loc[
            dm2["Template"].str.contains(m, regex=False, na=False), "Attribute"
        ].tolist()
    )

In [640]:
dm2 = pd.concat([dm2, templates], axis=0, ignore_index=True)

print(dm2.shape)

(634, 14)


In [641]:
dm2.sample(20)

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
297,DNA,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,TRUEHasIonizationSource,When hasIonizationSource = TRUE,,False,lensVoltagesUnit,,ValidValue,,Metabolomics Human,,"Sage Bionetworks,Proposed minimum metadata rel...",NaN,NaN,NaN
321,GIS,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,Zeiss LSM 980,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,TRUEUseReagent,When useReagent = TRUE,,,reagentContact,,ValidValue,,genotyping,,"Sage Bionetworks,ImmPort",NaN,NaN,NaN
510,The Health and Aging and Body Composition Stud...,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
592,pl,Measurement unit,NaN,False,NaN,NaN,validValue,NaN,NaN,MeasurementUnit,NaN,STRING,NaN,False
456,Pancreas,NaN,NaN,False,NaN,NaN,validValue,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624,Individual Human,Metadata template for Individual Human,NaN,True,"age,cohort,consentGroupID,countryCode,diagnosi...",NaN,dataType,NaN,NaN,Template,NaN,NaN,NaN,NaN
588,pg/mg creatinine,Measurement unit,NaN,False,NaN,NaN,validValue,NaN,NaN,MeasurementUnit,NaN,STRING,NaN,False


In [642]:
def clean_list_col(row):
    # cleanup list columns
    # split list
    # strip white space
    # only unique values
    # sorted alphabetically

    row = [y.strip() for y in row.split(",")]
    row = sorted(np.unique(row))
    row = ",".join(row)
    row = row.strip(",")

    return row


dm2[["Valid Values", "Template", "Ontology", "Source"]] = (
    dm2[["Valid Values", "Template", "Ontology", "Source"]]
    .fillna("")
    .applymap(clean_list_col)
)

In [643]:
dm2 = dm2.drop(
    index=dm2.loc[(dm2["Attribute"] == "f") | (
        dm2["Attribute"] == "F"),].index.tolist()
).reset_index(drop=True)

In [644]:
# Recode required columns and fix spelling mistakes
required_recoder = {"0.0": "False", "1.0": "True", "FASLSE": "False"}

dm2["Required"] = dm2["Required"].replace(required_recoder)

# dm2[~dm2["Required"].str.contains("True|False", regex=True)]

Last bit of cleanup


In [645]:
# Remove measurement technique dependency from biospecimen human
dm2.loc[dm2["Attribute"] == "Biospecimen human", "DependsOn"] = (
    dm2.loc[dm2["Attribute"] == "Biospecimen human", "DependsOn"]
    .values[0]
    .replace("measurementTechnique,", "")
)
bio_measure_technique_index = dm2.query(
    'Attribute == "measurementTechnique" and Parent == "Biospecimen human"'
)

if len(bio_measure_technique_index.index) > 0:
    dm2 = dm2.drop(
        index=bio_measure_technique_index.index[0]).reset_index(drop=True)

In [646]:
# Remove measurement technique dependency from biospecimen human
dm2.loc[dm2["Attribute"] == "Biospecimen human", "DependsOn"] = (
    dm2.loc[dm2["Attribute"] == "Biospecimen human", "DependsOn"]
    .values[0]
    .replace("specifyMeasurementTechnique,", "")
)
bio_measure_technique_index = dm2.query(
    'Attribute == "specifyMeasurementTechnique" and Parent == "Biospecimen human"'
)


if len(bio_measure_technique_index.index) > 0:
    dm2 = dm2.drop(
        index=bio_measure_technique_index.index[0]).reset_index(drop=True)

In [647]:
# Remove measurement technique dependency from biospecimen human
dm2.loc[dm2["Attribute"] == "Biospecimen human", "DependsOn"] = (
    dm2.loc[dm2["Attribute"] == "Biospecimen human", "DependsOn"]
    .values[0]
    .replace("OtherMeasurementTechnique,", "")
)
bio_measure_technique_index = dm2.query(
    'Attribute == "OtherMeasurementTechnique"')


if len(bio_measure_technique_index.index) > 0:
    dm2 = dm2.drop(
        index=bio_measure_technique_index.index[0]).reset_index(drop=True)

In [648]:
dm2.loc[dm2[dm2["Attribute"] == "visitCode"].index.values[0], "Valid Values"] = ""

dm2.loc[dm2[dm2["Attribute"] == "visitCode"].index.values[0],
        "Validation Rules"] = ""

Extra comma at beginning of valid values


# Validation Rules


In [649]:
mixed_attrs = [
    {"attribute": "tissueWeight", "val_type": "mixed float", "regex": "regex search"},
    {"attribute": "tissueVolume", "val_type": "mixed float", "regex": "regex search"},
    {"attribute": "specimenAge", "val_type": "mixed integer", "regex": "regex search"},
    {"attribute": "samplingAge", "val_type": "mixed integer", "regex": "regex search"},
    {"attribute": "age", "val_type": "mixed integer", "regex": "regex search"},
]

for ma in mixed_attrs:
    attribute = ma["attribute"]
    val_type = ma["val_type"]

    # get indexes for new validation rules based on attribute
    indexes = dm2[dm2["Attribute"] == attribute].index.tolist()

    for i in indexes:
        if val_type == "integer":
            first_part = "[0-9]+"
        elif val_type == "float":
            first_part = "^\d*?\.?\d$"
        elif val_type == "mixed integer":
            regex = "regex search"
            num_match = "^\d*?"
        elif val_type == "mixed float":
            regex = "regex search"
            num_match = "^\d*?\.?\d$"
            # All valid values are applicable
            new_string = (
                regex
                + num_match
                + "|"
                + ""
                + "|".join(dm2.loc[i, "Valid Values"].split(","))
            )

        dm2.loc[i, "Validation Rules"] = new_string

In [650]:
dm2["Validation Rules"].unique().tolist()

['',
 'regex search ([0-9]+\\.[0-9]*.)|([0-9]+)',
 'regex search^\\d*?\\.?\\d$|Not applicable|Not collected|Unknown',
 'regex search ([0-9]+)',
 'nan',
 nan]

## Building Dependencies


In [651]:
dependencies = {
    "specimenID": "matchAtLeastOne Biospecimenhuman.specimenID value",
    "individualID": "matchExactlyOne IndividualHuman.individualID set",
}

In [652]:
for k, v in dependencies.items():
    indexes = dm2[dm2["Attribute"] == k]["Validation Rules"].index.values
    dm2.loc[indexes, "Validation Rules"] = v

In [653]:
dm2.loc[3, "Valid Values"] = dm2.loc[3, "Valid Values"].replace(
    ",Whole Genome Sequencing", ""
)

# Create File Annotations Attributes


In [654]:
# base file annotations
base_file_annotations = {"resourceType": "",
                         "isReleased": False, "fileType": ""}

In [655]:
with pd.option_context("display.max_colwidth", None):
    display(dm2.query('Attribute == "visitCode"'))

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
177,visitCode,"Indicate which longitudinal visit for the individual the data comes from, provided by the data contributor's data dictionary",,True,,,,,Individual Human,,Sage Bionetworks,number,"Data Contributor,Sage Bionetworks",False


# Write out new data model


In [656]:
hard_coded_valid_values = [
    {
        "attribute": "visitCode",
        "valid_value": "1,2,Other,Unknown,Not collected,Not applicable",
    },
    {"attribute": "consentGroupID", "valid_value": "1,2,3"},
    {"attribute": "samplingAge", "valid_value": ""},
    {"attribute": "specimenAge", "valid_value": ""},
    {"attribute": "age", "valid_value": ""},
]

In [657]:
for h in hard_coded_valid_values:
    dm2.loc[
        dm2[dm2["Attribute"] == h["attribute"]].index.values[0], "Valid Values"
    ] = h["valid_value"]

In [658]:
checks = [c["attribute"] for c in hard_coded_valid_values]

with pd.option_context("display.max_colwidth", None):
    display(dm2.query(f"Attribute in @checks"))

,Attribute,Description,Valid Values,Required,DependsOn,DependsOn Component,Properties,Validation Rules,Template,Parent,Source,Type,Ontology,multivalue
6,age,"Age of the individual (age in years of the individual at first recorded study event (enrollment, visit, observation, sample collection, survey completion, etc.)",,True,,,,regex search^\d*?\.?\d$|Not applicable|Not collected|Unknown,Individual Human,,Sage Bionetworks,string,Sage Bionetworks,False
18,consentGroupID,"Indicate the consent group for the individual, provided by the data contributor's data dictionary","1,2,3",True,,,,,Individual Human,,Sage Bionetworks,string,"Data Contributor,Sage Bionetworks",False
106,samplingAge,"The calculated age of the sample, measurement is determined or coded by the data contributor.",,True,,,,regex search^\d*?\.?\d$|Not applicable|Not collected|Unknown,"Biospecimen human,Biospecimen nonHuman",,Sage Bionetworks,string,Sage Bionetworks,False
150,specimenAge,The subject's age at the time of specimen extraction is the individual's age (since birth) at the time a given specimen was extracted. Measured as age in years.,,True,,,,regex search^\d*?\.?\d$|Not applicable|Not collected|Unknown,"Biospecimen human,Biospecimen nonHuman",,Sage Bionetworks,string,Sage Bionetworks,False
177,visitCode,"Indicate which longitudinal visit for the individual the data comes from, provided by the data contributor's data dictionary","1,2,Other,Unknown,Not collected,Not applicable",True,,,,,Individual Human,,Sage Bionetworks,number,"Data Contributor,Sage Bionetworks",False


In [659]:
import yaml

with open("./local_configs/notebook_config.yaml", "r") as f:
    config = yaml.safe_load(f)

# paths to import files
schematic_config = config["paths"]["schematic"]
csv_model = config["file_names"]["csv_model"]
json_model = config["file_names"]["json_model"]

print(
    "Schematic config: ",
    schematic_config,
    "\n",
    "CSV model: ",
    csv_model,
    "\n",
    "JSON LD Model: ",
    json_model,
)

Schematic config:  ./config.yml 
 CSV model:  EL.data.model.csv 
 JSON LD Model:  EL.data.model.jsonld


In [660]:
# Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,Module,Type,Ontology,UsedIn

In [661]:
print(dm2.columns.tolist())

['Attribute', 'Description', 'Valid Values', 'Required', 'DependsOn', 'DependsOn Component', 'Properties', 'Validation Rules', 'Template', 'Parent', 'Source', 'Type', 'Ontology', 'multivalue']


In [662]:
dm2 = dm2.rename({"Template": "UsedIn"}, axis=1)

In [663]:
print(dm2.shape)

dm2 = (
    dm2.fillna("").drop_duplicates(subset=["Attribute"]).reset_index(drop=True)
)  # 'DependsOn', 'Properties',

display(dm2.shape)

(632, 14)


(612, 14)

In [664]:
from utils import utils

In [ ]:
dm_old = utils.load_and_backup_dm("../EL.data.model.csv", "../backups")
dm2 = dm2.replace(r"^\s*$", np.nan, regex=True)
new_dm = pd.concat([dm2, dm_old])

In [ ]:
test_dm = (
    new_dm.fillna("")
    .groupby("Attribute")
    .agg(lambda x: ",".join(set(x.astype(str))))
    .reset_index()
)
test_dm = test_dm.applymap(lambda x: x.strip(","))
test_dm = test_dm.replace(r"^\s*$", np.nan, regex=True)
test_dm[["Module", "Properties", "Parent", "Type", "Ontology", "Source"]] = test_dm[
    ["Module", "Properties", "Parent", "Type", "Ontology", "Source"]
].fillna("Unspecified")

test_dm["Properties"] = test_dm["Properties"].apply(
    lambda x: x[0].upper() + x[1:])
test_dm["Type"] = test_dm["Type"].str.upper()
more_recoding = {"ValidValue,MeasurementUnit": "MeasurementUnit"}
test_dm["Parent"] = test_dm["Parent"].replace(more_recoding)
test_dm.loc[test_dm["Attribute"] == "visitCode", "Valid Values"] = ""

In [695]:
test_dm["Validation Rules"] = test_dm["Validation Rules"].replace(
    "regex search^\d*?\.?\d$|Unknown|Not collected|Not applicable,regex search^\d*?\.?\d$|Not applicable|Not collected|Unknown",
    "regex search^\d*?\.?\d$|Unknown|Not collected|Not applicable",
)

In [696]:
# write out data model into csv
test_dm.to_csv(csv_model)

In [697]:
# Convert CSV to JSON LD
print(f'schematic schema convert {csv_model} --output_jsonld {json_model}')

!schematic schema convert {csv_model} --output_jsonld {json_model}

schematic schema convert EL.data.model.csv --output_jsonld EL.data.model.jsonld
Starting schematic...
Done adding requirements and value ranges to attributes
The Data Model was created and saved to 'EL.data.model.jsonld' location.
